In [1]:
# TODO:
# Create a data-augmentor + data-preprocessing class
# Using cv2 not pil and maybe using https://github.com/aleju/imgaug
# Will ensure that pre-processing + augmentation standardised across frameworks
# Also maybe PIL is bottlenecking pytorch?

# Notes
# http://corochann.com/install-chainer-1401.html#Install_NCCL

In [2]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers, cuda, dataset, training
from chainer.training import extensions, updaters, StandardUpdater
from chainer.dataset import concat_examples
from chainer.links import ResNet50Layers
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from PIL import Image
import random
#import cv2
from common.utils import *

In [3]:
# Performance Improvement
# 1. Auto-tune
# This adds very little now .. not sure if True by default?
chainer.global_config.autotune = True

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Chainer: ", chainer.__version__)
print("CuPy: ", chainer.cuda.cupy.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())
CPU_COUNT = multiprocessing.cpu_count()
print("CPUs: ", CPU_COUNT)

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Chainer:  3.4.0
CuPy:  2.4.0
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21
CPUs:  12


In [5]:
# Set "num_gpus"
NUM_GPUS = 2

In [6]:
MULTI_GPU=NUM_GPUS>1
DEVICES=tuple(list(range(NUM_GPUS)))
if MULTI_GPU:
    from cupy.cuda import nccl  # Test that nccl works for multi-gpu

In [7]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001  # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
#BATCHSIZE = 64*NUM_GPUS
BATCHSIZE = 64  # Chainer auto scales batch
IMAGENET_RGB_MEAN =  np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_RGB_SD =  np.array([0.229, 0.224, 0.225], dtype=np.float32)
TOT_PATIENT_NUMBER = 30805  # From data

In [8]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [9]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 666 ms, sys: 225 ms, total: 890 ms
Wall time: 889 ms


In [10]:
#####################################################################################################
## Data Loading

In [11]:
class XrayData(dataset.DatasetMixin):
    def __init__(self, patient_ids, img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, augmentation=None):
        # Read labels-csv
        df = pd.read_csv(lbl_file)
        
        # Split labels on unfiltered data
        df_label = df['Finding Labels'].str.split(
            '|', expand=False).str.join(sep='*').str.get_dummies(sep='*')
        
        # Filter by patient-ids (both)
        df_label['Patient ID'] = df['Patient ID']
        df_label = df_label[df_label['Patient ID'].isin(patient_ids)]
        df = df[df['Patient ID'].isin(patient_ids)]
        
        # Remove unncessary columns
        df_label.drop(['Patient ID','No Finding'], axis=1, inplace=True)  
        
        # List of images (full-path)
        self.img_locs =  df['Image Index'].map(lambda im: os.path.join(img_dir, im)).values
        # One-hot encoded labels (float32 for BCE loss)
        self.labels = df_label.values
        
        # Processing
        self.augmentation = augmentation
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
        
    def __len__(self):
        return len(self.img_locs)   
    
    def get_example(self, idx):
        im_file = self.img_locs[idx]
        # RGB Image
        im_rgb = Image.open(im_file).convert('RGB')
        im_rgb = _apply_data_preprocessing(im_rgb)
        label = self.labels[idx]
        if self.augmentation is not None:
            im_rgb = _apply_data_augmentation(im_rgb)
        return np.array(im_rgb, dtype=np.float32), np.array(label, dtype=np.int32)
    
def _apply_data_preprocessing(rgb_im, w=WIDTH, h=HEIGHT, 
                              rgb_m=IMAGENET_RGB_MEAN, rgb_sd=IMAGENET_RGB_SD):
    # REDO Method!
    # https://github.com/keras-team/keras/blob/master/keras/preprocessing/image.py#L325
    # Resize
    rgb_im = rgb_im.resize((w, h), Image.NEAREST)
    # Array
    im = np.asarray(rgb_im, dtype=np.float32)
    # (w, h, c) to (c, h, w)
    im = im.transpose(2, 0, 1)
    # Normalise image with imagenet-values
    # This is torch-model normalisation
    # Ref: https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py
    im = im / 255.
    im = (im - rgb_m[:, None, None]) / rgb_sd[:, None, None]
    return im

def _apply_data_augmentation(x):
    # REDO Method!
    # Random horizontal flip
    # col_axis = 2
    if random.randint(0, 1):
        x = x.swapaxes(2, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, 2)
    # Random rotation
    # ...
    # Random crop/zoom
    # ...
    return x

In [12]:
# Training / Valid / Test split (70% / 10% / 20%)
train_set, other_set = train_test_split(
    range(1,TOT_PATIENT_NUMBER+1), train_size=0.7, test_size=0.3, shuffle=False)
valid_set, test_set = train_test_split(other_set, train_size=1/3, test_size=2/3, shuffle=False)
print("train:{} valid:{} test:{}".format(
    len(train_set), len(valid_set), len(test_set)))

train:21563 valid:3080 test:6162


In [13]:
train_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=train_set, augmentation=True)
valid_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=valid_set, augmentation=False)
test_dataset  = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=test_set, augmentation=False)

Loaded 87306 labels and 87306 images
Loaded 7616 labels and 7616 images
Loaded 17198 labels and 17198 images


In [14]:
#####################################################################################################
## Helper Functions

In [15]:
class chexnet(ResNet50Layers):
    
    def __init__(self, pretrained_model="auto", basemodel_out='pool5', out_features=CLASSES):
        super(chexnet, self).__init__(pretrained_model)
        self._basemodel_out = basemodel_out
        in_features = 2048
        with self.init_scope():
            self.classifier = L.Linear(in_features, out_features)
    
    def __call__(self, x, **kwargs):
        h = super(chexnet, self).__call__(x, layers=[self._basemodel_out], **kwargs)[self._basemodel_out]
        h = self.classifier(h)
        return h

In [16]:
def get_symbol(model_name='densenet121', multi_gpu=MULTI_GPU):
    if model_name == 'resnet50':
        m = chexnet()
    elif model_name == 'densenet121':
        # https://github.com/chainer/chainer/issues/4426
        raise ValueError("Densenet is not yet officially implemented in Chainer")
    else:
        raise ValueError("Unknown model-name")
    # CUDA
    if not multi_gpu:
        print("One GPU")
        chainer.cuda.get_device(0).use()  # Make a specified GPU current
        m.to_gpu()  
    return m

In [17]:
def init_symbol(sym, lr=LR):
    opt = optimizers.Adam(alpha=lr, beta1=0.9, beta2=0.999)
    opt.setup(sym)
    return opt

In [18]:
def compute_roc_auc(data_gt, data_pd, full=True, classes=CLASSES):
    roc_auc = []
    for i in range(classes):
        roc_auc.append(roc_auc_score(data_gt[:, i], data_pd[:, i]))
    print("Full AUC", roc_auc)
    roc_auc = np.mean(roc_auc)
    return roc_auc

In [19]:
def lossfun(x, t):
    return F.sigmoid_cross_entropy(x, t)

In [20]:
%%time
# Load symbol
predictor = get_symbol(model_name='resnet50')
chexnet_sym = L.Classifier(predictor, lossfun=lossfun)
chexnet_sym.compute_accuracy = False

CPU times: user 381 ms, sys: 80.1 ms, total: 461 ms
Wall time: 461 ms


In [21]:
%%time
# Load optimiser
optimizer = init_symbol(chexnet_sym)

CPU times: user 718 µs, sys: 0 ns, total: 718 µs
Wall time: 725 µs


In [22]:
# Data-iterators
# Open-CV runs on GPU by default!
# Weird-bug to set shared_mem (set random value and adjust)
if MULTI_GPU:
    train_iters = [
        chainer.iterators.MultiprocessIterator(
            train_dataset, BATCHSIZE, shared_mem=700000, n_prefetch=10, n_processes=CPU_COUNT) 
        for i in chainer.datasets.split_dataset_n_random(train_dataset, len(DEVICES))]
else:
    train_iter = chainer.iterators.MultiprocessIterator(
        train_dataset, BATCHSIZE, shared_mem=700000, n_prefetch=10, n_processes=CPU_COUNT)

In [23]:
# These can have a higher batch-size than train since no grads stored
valid_iter = chainer.iterators.MultiprocessIterator(
    valid_dataset, BATCHSIZE, shared_mem=700000, repeat=False, shuffle=False, n_prefetch=10, n_processes=CPU_COUNT)
test_iter = chainer.iterators.MultiprocessIterator(
    test_dataset, BATCHSIZE, shared_mem=700000, repeat=False, shuffle=False, n_prefetch=10, n_processes=CPU_COUNT)

In [24]:
# MultiprocessParallelUpdater requires NCCL.
# https://github.com/nvidia/nccl#build--run
if MULTI_GPU:
    updater = updaters.MultiprocessParallelUpdater(train_iters, optimizer, devices=DEVICES)
else:
    updater = StandardUpdater(train_iter, optimizer, device=0)
val_interval = (1, 'epoch')

/anaconda/envs/py35/lib/python3.5/site-packages/chainer/training/updaters/multiprocess_parallel_updater.py:131: UserWarning: optimizer.eps is changed to 2e-08 by MultiprocessParallelUpdater for new batch size.
  format(optimizer.eps))


In [25]:
trainer = training.Trainer(updater, stop_trigger=(EPOCHS, 'epoch'))
trainer.extend(extensions.Evaluator(valid_iter, chexnet_sym, device=DEVICES[0]), trigger=val_interval)
trainer.extend(extensions.LogReport(trigger=val_interval))
trainer.extend(extensions.PrintReport(['epoch', 'iteration', 'main/loss', 'validation/main/loss']), trigger=val_interval)
trainer.extend(extensions.ProgressBar(update_interval=500))

In [26]:
#+-----------------------------------------------------------------------------+
#| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
#|-------------------------------+----------------------+----------------------+
#| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
#| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
#|===============================+======================+======================|
#|   0  Tesla P100-PCIE...  Off  | 0000BC4B:00:00.0 Off |                    0 |
#| N/A   56C    P0   196W / 250W |  16216MiB / 16276MiB |     99%      Default |
#+-------------------------------+----------------------+----------------------+
#|   1  Tesla P100-PCIE...  Off  | 0000D419:00:00.0 Off |                    0 |
#| N/A   57C    P0   174W / 250W |   6651MiB / 16276MiB |    100%      Default |
#+-------------------------------+----------------------+----------------------+

In [27]:
%%time
# 1 GPU = 3.1864 iters/sec
# Wall time: 36m46s
# 2 GPU = 2.6848 iters/sec
# Wall time: 41min9s
trainer.run()

     total [###...............................................]  7.33%
this epoch [##################................................] 36.65%
       500 iter, 0 epoch / 5 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [#######...........................................] 14.66%
this epoch [####################################..............] 73.31%
      1000 iter, 0 epoch / 5 epochs
    2.8483 iters/sec. Estimated time to finish: 0:34:03.616602.
epoch       iteration   main/loss   validation/main/loss
1           1365        0.15378     0.141193              
     total [##########........................................] 21.99%
this epoch [####..............................................]  9.96%
      1500 iter, 1 epoch / 5 epochs
    2.7364 iters/sec. Estimated time to finish: 0:32:24.479975.
     total [##############....................................] 29.32%
this epoch [#######################...........................] 46.61%
      2000 iter, 1 epoc

In [28]:
#####################################################################################################
## Test CheXNet

In [29]:
# Can load model from checkpoint
#model = get_symbol(model_name='resnet50')
# Load the saved paremeters into the instance
#serializers.load_npz('my_mnist.model', model)

In [30]:
%%time
y_truth = test_dataset.labels
y_guess = []
test_iter.reset()
with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    for test_batch in test_iter:
        # Data
        x_test, y_test = concat_examples(test_batch, device=DEVICES[0])
        # Prediction (need to apply sigmoid to turn into probability)
        pred = cuda.to_cpu(F.sigmoid(predictor(x_test)).data)
        # Collect results
        y_guess.append(pred)           
# Concatenate
y_guess = np.concatenate(y_guess, axis=0)

CPU times: user 27 s, sys: 6.86 s, total: 33.9 s
Wall time: 33 s


In [31]:
# Test AUC: 0.7993
print("Test AUC: {0:.4f}".format(compute_roc_auc(y_truth, y_guess)))

Full AUC [0.7898239277954502, 0.8376775446313578, 0.7568095698794695, 0.8524100729679737, 0.867159043399087, 0.9078994670721408, 0.7350985953693724, 0.8017389668333064, 0.583063570088059, 0.8163193902518544, 0.7154199872264138, 0.771522169766665, 0.6983676685804345, 0.8591604665524035]
Test AUC: 0.7852
